In [30]:
import cv2
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'cv2'

In [28]:
import sys
print(sys.executable)


/usr/local/opt/python@3.10/bin/python3.10


In [51]:
import sys
print(sys.executable)

import cv2
print("OpenCV version:", cv2.__version__)


/usr/local/opt/python@3.10/bin/python3.10


ModuleNotFoundError: No module named 'cv2'

In [ ]:
model = YOLO('yolov8s.pt')

In [ ]:

import cv2
from ultralytics import YOLO
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, clear_output
import time

# Load your trained model
model = YOLO('best.pt')  # Make sure your model file is in the same directory

# Initialize webcam
def init_webcam():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        # Try different camera indices if 0 doesn't work
        cap = cv2.VideoCapture(1)
        if not cap.isOpened():
            print("Error: Could not open webcam")
            return None
    return cap

# Function to process frame and detect ESP32
def detect_objects(frame):
    # Convert BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Run detection
    results = model(rgb_frame, conf=0.5)
    
    # Process results
    detected_frame = rgb_frame.copy()
    pil_image = Image.fromarray(detected_frame)
    draw = ImageDraw.Draw(pil_image)
    
    for result in results:
        if result.boxes is not None:
            boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes
            confidences = result.boxes.conf.cpu().numpy()  # Confidence scores
            classes = result.boxes.cls.cpu().numpy()  # Class IDs
            
            for i, box in enumerate(boxes):
                x1, y1, x2, y2 = map(int, box)
                confidence = confidences[i]
                class_id = int(classes[i])
                
                # Draw bounding box
                draw.rectangle([x1, y1, x2, y2], outline="green", width=2)
                
                # Draw label
                label = f"ESP32: {confidence:.2f}"
                draw.text((x1, y1 - 20), label, fill="green")
    
    return pil_image

# Main function for real-time detection
def run_realtime_detection():
    cap = init_webcam()
    if cap is None:
        return
    
    print("Starting real-time ESP32 detection...")
    print("Press the stop button in Jupyter to terminate")
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame")
                break
            
            # Detect objects
            detected_image = detect_objects(frame)
            
            # Display in Jupyter
            clear_output(wait=True)
            display(detected_image)
            
            # Add a small delay to control frame rate
            time.sleep(0.1)
            
    except KeyboardInterrupt:
        print("Stopping detection...")
    
    finally:
        cap.release()
        print("Webcam released")

# Run the detection
run_realtime_detection()

In [ ]:
image_path = 'https://ultralytics.com/images/bus.jpg'
results = model(image_path, conf=0.5)
results[0].show()
